In [1]:
import cv2
from tqdm import tqdm
import os
import glob

In [2]:
frames_dir = 'preprocessed-frames'
output_dir = 'motion-detection-frames'

In [3]:
out = cv2.VideoWriter("motion-detection-output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 15, (3420,1910))


In [4]:
for i in tqdm(range(len(glob.glob1(frames_dir, "*.jpg")) - 1)):
    prev_frame = cv2.imread(f"{frames_dir}/frame{i}.jpg")
    curr_frame = cv2.imread(f"{frames_dir}/frame{i + 1}.jpg")

    diff = cv2.absdiff(curr_frame, prev_frame)
    _, binary_diff = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    clean_diff = cv2.morphologyEx(binary_diff, cv2.MORPH_CLOSE, kernel)
    
    out.write(clean_diff)
    cv2.imwrite(f"{output_dir}/frame{i}.jpg", clean_diff)    

    # flow = cv2.calcOpticalFlowFarneback(prev_frame, curr_frame, None,
    #                                 0.5, 3, 15, 3, 5, 1.2, 0)
    # magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
out.release()

100%|██████████| 749/749 [01:04<00:00, 11.64it/s]
